<a href="https://colab.research.google.com/github/mars-ef/Trustpilot_Scraper/blob/main/Trustpilot_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#Analytics with Adam video on how to use https://www.youtube.com/watch?v=VD0wIiLVSRA

from time import sleep
import requests
from google.colab import files
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

brand = [] #Same as company
users = []
ratings = []
locations = []
experiences = [] #Date of experience being reviewed
dates = [] #Date review posted or updated
reviews = [] #Content of review
replies = [] #Whether business replied
reply_dates = [] #Date business replied
from_page = 1
to_page = 1
company = 'www.revolut.com' #E.g. for https://www.trustpilot.com/review/wise.com to sort default take 'wise.com'. To filter by most recent include 'wise.com?sort=recency'

for i in range(from_page, to_page + 1):
    result = requests.get(f"https://www.trustpilot.com/review/{company}?page={i}") #for most recent change to '{company}page={i}&sort=recency'
    soup = BeautifulSoup(result.content, 'html.parser')

    soup2list(soup.find_all('span', {'class', 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), users)
    soup2list(soup.find_all('div', {'class', 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), locations)
    soup2list(soup.find_all('div', {'class', 'styles_reviewHeader__iU9Px'}), dates)
    soup2list(soup.find_all('div', {'class', 'styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')
    soup2list(soup.find_all('div', {'class', 'styles_reviewContent__0Q2Tg'}), reviews)
    soup2list(soup.find_all('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17', 'data-service-review-date-of-experience-typography': 'true'}), experiences)

    # Adds business replies and dates, makes sure records added even if no business reply
    replies_ = soup.find_all('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 typography_weight-heavy__E1LTj styles_replyCompany__ro_yX'})
    dates_ = soup.find_all('time', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_replyDate__Iem0_'})

    for review in soup.find_all('article', {'class': 'styles_reviewCard__hcAvl'}):
        reply_tag = review.find('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 typography_weight-heavy__E1LTj styles_replyCompany__ro_yX'})
        date_tag = review.find('time', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_replyDate__Iem0_'})

        if reply_tag:
            replies.append(1)
            replies_.remove(reply_tag)
            reply_dates.append(date_tag.get_text())
            dates_.remove(date_tag)
        else:
            replies.append(0)
            reply_dates.append('')

   # To avoid throttling
    sleep(1)

# Remove extra words from various columns before adding to DataFrame
def remove_extra_words(text, words_to_remove):
    for word in words_to_remove:
        text = text.replace(word, '').strip()
    return text

dates = [remove_extra_words(date, ['Updated', 'Verified', 'Redirected']) for date in dates]
content = [remove_extra_words(content, ['Date of experience:.*']) for content in reviews]
experiences = [remove_extra_words(experience, ['Date of experience:', 'Updated', 'Verified', 'Redirected']) for experience in experiences]
reply_dates = [remove_extra_words(reply_date, ['Updated', 'Verified', 'Redirected']) for reply_date in reply_dates]

# Create the DataFrame
review_data = pd.DataFrame({
    'brand': company,
    'username': users,
    'location': locations,
    'date': dates,
    'content': content,
    'rating': ratings,
    'date_of_experience': experiences,
    'reply_from_business': replies,
    'reply_date': reply_dates
})

# Removes 'invited' from 'date' and adds the invited column
review_data['invited'] = 0
for index, row in review_data.iterrows():
    if 'Invited' in row['date']:
        review_data.loc[index, 'invited'] = 1
        review_data.loc[index, 'date'] = row['date'].replace('Invited', '').strip()

#Converts the data in 'date' and 'reply_date' to Yyyy-Mm-Dd format
def convert_relative_date(date_str):
  if date_str == 'Just now':
    return datetime.now().strftime('%Y-%m-%d')
  elif date_str == 'An hour ago':
    return (datetime.now() - timedelta(hours=1)).strftime('%Y-%m-%d')
  elif date_str == 'A day ago':
    return (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
  elif 'days ago' in date_str:
    days = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
  elif 'hours ago' in date_str:
    hours = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(hours=hours)).strftime('%Y-%m-%d')
  elif 'minutes ago' in date_str:
    minutes = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(minutes=minutes)).strftime('%Y-%m-%d')
  else:
    try:
      return datetime.strptime(date_str, '%B %d, %Y').strftime('%Y-%m-%d')
    except ValueError:
      return date_str

review_data['date'] = review_data['date'].apply(convert_relative_date)
review_data['date_of_experience'] = review_data['date_of_experience'].apply(convert_relative_date)
review_data['reply_date'] = review_data['reply_date'].apply(convert_relative_date)

In [20]:
review_data

,brand,username,location,date,content,rating,date_of_experience,reply_from_business,reply_date,invited
0,www.revolut.com,Moeno Mokati,FI,2024-09-24,"When issues pop upWhen issues pop up, it’s rea...",5,2024-09-24,0,,1
1,www.revolut.com,Uma Purohit,GB,2024-09-24,One card does it allWhilst travelling I find i...,5,2024-09-24,0,,1
2,www.revolut.com,Nnabiko Augustine Chime,GB,2024-09-24,I'm extremely satisfied with RevolutI'm extrem...,5,2024-09-24,0,,1
3,www.revolut.com,Dave D,GB,2024-09-25,An example to the rest in how to do it We have...,5,2024-09-25,0,,0
4,www.revolut.com,Mārtiņš Kronbergs,LV,2024-09-24,"For Ultra planFor Ultra plan, better to have u...",4,2024-09-24,0,,1
5,www.revolut.com,Michael Germitsiotis,GR,2024-09-24,Excellent!I'm very satisfied with both the day...,5,2024-09-24,0,,0
6,www.revolut.com,Hung Lieu,NL,2024-09-29,Recently used the product for currency…Recentl...,2,2024-09-29,0,,0
7,www.revolut.com,Sylvia Ugah,GB,2024-09-24,My experience with revolut has been…My experie...,5,2024-09-24,0,,1
8,www.revolut.com,Ruth Hamblin,GB,2024-09-25,I rely on using my Revolut card in the…I rely ...,5,2024-09-24,0,,1
9,www.revolut.com,Karen Pugh,AU,2024-09-24,"Great to use aboardGreat to use aboard, althou...",4,2024-09-24,0,,1


In [ ]:
# Convert the DataFrame to a CSV file and download it
review_data.to_csv('reviews.csv', index=False)
files.download('reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>